<a href="https://colab.research.google.com/github/thogood212/HnM_recommendations/blob/main/HnM_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#H&M Kaggle 추천시스템
(https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/overview)

##**개요**

H&M Group 은 53개의 온라인 시장과 약 4,850개의 매장을 보유한 브랜드 및 비즈니스 제품군입니다.

우리의 온라인 상점은 쇼핑객이 탐색할 수 있는 광범위한 제품을 제공합니다. 그러나 선택의 폭이 너무 넓으면 고객이 자신이 관심을 갖는 것 또는 찾고 있는 것을 빨리 찾지 못하고 궁극적으로 구매하지 않을 수도 있습니다. 

쇼핑 경험을 향상시키기 위해서는 제품 추천이 핵심입니다. 더 중요한 것은 고객이 올바른 선택을 하도록 돕는 것이 반품을 줄이고 운송으로 인한 배출량을 최소화하기 때문에 지속 가능성에 긍정적인 영향을 미친다는 것입니다.

이 대회에서 H&M Group은 고객 및 제품 메타 데이터뿐만 아니라 이전 거래의 데이터를 기반으로 제품 권장 사항을 개발하도록 초대합니다. 사용 가능한 메타 데이터는 의류 유형 및 고객 연령과 같은 간단한 데이터에서 제품 설명의 텍스트 데이터, 의류 이미지의 이미지 데이터에 이르기까지 다양합니다.

어떤 정보가 유용할 수 있는지에 대한 선입견은 없습니다. 범주형 데이터 유형 알고리즘을 조사하거나 NLP 및 이미지 처리 딥 러닝에 대해 알아보려는 경우 이는 사용자에게 달려 있습니다.

---

##**데이터**

1. images - 이번 구현에서는 해당 이미지 데이터를 사용하지 않았습니다.
2. Articles.csv - 구매할 수 있는 각 항목 에 대한 자세한 메타데이터
3. customer.csv - 데이터세트의 각 메타 customer_id데이터
4. sample_submission.csv - 올바른 형식의 샘플 제출 파일
5. transaction_train.csv - 각 날짜에 대한 각 고객의 구매 및 추가 정보로 구성된 훈련 데이터입니다.  
 중복 행은 동일한 항목의 여러 구매에 해당합니다.

---

##**목표**

이 프로젝트의 목표는 훈련데이터 기간 직후 7일 기간 동안 각 고객(customer_id)이 구매할 article_id를 예측하는 것입니다.







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset

import pandas as pd
import numpy as np
import os

##일지
1. 22.08.18 - 데이터 EDA

##EDA

###Train

In [3]:
#데이터 불러오기

train = pd.read_csv("/content/drive/MyDrive/hnm_recommendations/transactions_train.csv.zip")
articles = pd.read_csv("/content/drive/MyDrive/hnm_recommendations/articles.csv.zip")
customers = pd.read_csv("/content/drive/MyDrive/hnm_recommendations/customers.csv.zip")

In [4]:
#훈련데이터 확인하기
'''
t_dat = 데이터 기록일자 = 2018-09-20부터 2020-09-22까지 데이터
customer_id = 고객 id = 1362281명의 고객 데이터
article_id = 구매물품id = 104547개 물품id
price = 가격 = 
sales_channel_id = 구매channel
'''
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [13]:
train.nunique()

t_dat                   734
customer_id         1362281
article_id           104547
price                  9857
sales_channel_id          2
dtype: int64

In [15]:
print(min(train['t_dat']),max(train['t_dat']))
print(min(train['price']),max(train['price']))

2018-09-20 2020-09-22
1.694915254237288e-05 0.5915254237288136


###Article

In [5]:
#article 데이터 확인하기
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [16]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [18]:
subtitle = list(articles.columns[-7:])
articles[subtitle]

,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...
105537,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


###customers

In [21]:
#customers 데이터 확인하기
customers.head(10)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


In [20]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB
